In [ ]:
import nltk
import re
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Data Collection - Task 1 - Sheet1.csv to Data Collection - Task 1 - Sheet1.csv


In [ ]:
import io
import pandas as pd

df = pd.read_csv(io.BytesIO(uploaded['Data Collection - Task 1 - Sheet1.csv']))

In [ ]:
# Remove rows with missing values in essential columns
df.columns = df.columns.str.strip() # Standardize column names first
df = df.dropna(subset=["Comments", "Reviews", "Platform"])

In [ ]:
#Clean text data (remove extra spaces, lowercase, etc.)
# Standardize column names first
df.columns = df.columns.str.strip()

df["Comments"] = df["Comments"].str.strip().str.lower()
df["Reviews"] = df["Reviews"].str.strip().str.lower()
df["Platform"] = df["Platform"].str.strip().str.lower() # Accessing 'Platform' without the trailing space
print(df.head())
print(df.tail())

                                            Comments   Reviews   Platform  \
1   ai is geting scary… can’t trust anything anymore  negative    youtube   
2                      this dude is super unlikeable  negative    youtube   
3                                  sick sick sick! 😡  negative  instagram   
4  😑such an average movie, not worth the hype at all  negative    youtube   
5                    nonsense...absolutely not true.  negative  instagram   

   Unnamed: 3 Unnamed: 4  
1         NaN        NaN  
2         NaN        NaN  
3         NaN   Negative  
4         NaN    Neutral  
5         NaN   Positive  
                                               Comments   Reviews   Platform  \
1236  this is a great platform for getting feedback ...  positive   facebook   
1237  the sunflare in this photo adds a beautiful ar...  positive  instagram   
1238  your new baby is the cutest thing i have ever ...  positive  instagram   
1239              i learned something new today thanks!

/tmp/ipython-input-3473612563.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Comments"] = df["Comments"].str.strip().str.lower()
/tmp/ipython-input-3473612563.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Reviews"] = df["Reviews"].str.strip().str.lower()
/tmp/ipython-input-3473612563.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.

In [ ]:
#Remove duplicates
df = df.drop_duplicates()


In [ ]:
#Reset index after cleaning
df = df.reset_index(drop=True)
print(df.head())
print(df.tail())

                                            Comments   Reviews   Platform  \
0   ai is geting scary… can’t trust anything anymore  negative    youtube   
1                      this dude is super unlikeable  negative    youtube   
2                                  sick sick sick! 😡  negative  instagram   
3  😑such an average movie, not worth the hype at all  negative    youtube   
4                    nonsense...absolutely not true.  negative  instagram   

   Unnamed: 3 Unnamed: 4  
0         NaN        NaN  
1         NaN        NaN  
2         NaN   Negative  
3         NaN    Neutral  
4         NaN   Positive  
                                               Comments   Reviews   Platform  \
1234  this is a great platform for getting feedback ...  positive   facebook   
1235  the sunflare in this photo adds a beautiful ar...  positive  instagram   
1236  your new baby is the cutest thing i have ever ...  positive  instagram   
1237              i learned something new today thanks!

In [ ]:
#Final cleaned dataset
print(df.head())
print(df.tail())
print("\nShape after preprocessing:", df.shape)

                                            Comments   Reviews   Platform
0   ai is geting scary… can’t trust anything anymore  negative    youtube
1                      this dude is super unlikeable  negative    youtube
2                                  sick sick sick! 😡  negative  instagram
3  😑such an average movie, not worth the hype at all  negative    youtube
4                    nonsense...absolutely not true.  negative  instagram
                                               Comments   Reviews   Platform
3360  our office is celebrating its annual awards an...  positive   facebook
3361  your new coffee shop is the perfect place for ...  positive  instagram
3362  this is a great platform for finding and shari...  positive   facebook
3363              i learned something new today thanks!  positive   facebook
3364        i learned a lot from this thread thank you.  positive    twitter

Shape after preprocessing: (3365, 3)


In [ ]:
#Standardize column names (remove spaces/newlines)
df.columns = df.columns.str.strip()

In [ ]:
#Simple stopwords list (offline, no downloads)
stop_words = set([
    "the","a","an","is","it","this","that","and","or","in","on","for","of","to","with",
    "was","were","be","been","am","are","as","at","by","from","but","so","if","then",
    "too","very","just","can","will","do","does","did"
])
print(df.head())
print(df.tail())

                                            Comments   Reviews   Platform
0   ai is geting scary… can’t trust anything anymore  negative    youtube
1                      this dude is super unlikeable  negative    youtube
2                                  sick sick sick! 😡  negative  instagram
3  😑such an average movie, not worth the hype at all  negative    youtube
4                    nonsense...absolutely not true.  negative  instagram
                                               Comments   Reviews   Platform
3360  our office is celebrating its annual awards an...  positive   facebook
3361  your new coffee shop is the perfect place for ...  positive  instagram
3362  this is a great platform for finding and shari...  positive   facebook
3363              i learned something new today thanks!  positive   facebook
3364        i learned a lot from this thread thank you.  positive    twitter


In [ ]:
import string

# Preprocessing function
def preprocess_text(text):
    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Tokenize and remove stopwords
    tokens = [word for word in text.split() if word not in stop_words]
    return " ".join(tokens)
    # Apply stemming preprocessing
df["Stemmed_Comments"] = df["Comments"].apply(preprocess_text)

print(df.head())
print(df.tail())

                                            Comments   Reviews   Platform  \
0   ai is geting scary… can’t trust anything anymore  negative    youtube   
1                      this dude is super unlikeable  negative    youtube   
2                                  sick sick sick! 😡  negative  instagram   
3  😑such an average movie, not worth the hype at all  negative    youtube   
4                    nonsense...absolutely not true.  negative  instagram   

                                Stemmed_Comments  
0  ai geting scary… can’t trust anything anymore  
1                          dude super unlikeable  
2                               sick sick sick 😡  
3         😑such average movie not worth hype all  
4                    nonsenseabsolutely not true  
                                               Comments   Reviews   Platform  \
3360  our office is celebrating its annual awards an...  positive   facebook   
3361  your new coffee shop is the perfect place for ...  positive  inst

In [ ]:
# Function to remove emojis
def remove_emojis(text):
    text = str(text)
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags
        u"\U00002700-\U000027BF"  # dingbats
        u"\U0001F900-\U0001F9FF"  # supplemental symbols
        u"\U00002600-\U000026FF"  # miscellaneous symbols
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)

# Example usage
df["No_Emoji_Comments"] = df["Comments"].apply(remove_emojis)

print(df.head())


                                            Comments   Reviews   Platform  \
0   ai is geting scary… can’t trust anything anymore  negative    youtube   
1                      this dude is super unlikeable  negative    youtube   
2                                  sick sick sick! 😡  negative  instagram   
3  😑such an average movie, not worth the hype at all  negative    youtube   
4                    nonsense...absolutely not true.  negative  instagram   

                                Stemmed_Comments  \
0  ai geting scary… can’t trust anything anymore   
1                          dude super unlikeable   
2                               sick sick sick 😡   
3         😑such average movie not worth hype all   
4                    nonsenseabsolutely not true   

                                  No_Emoji_Comments  
0  ai is geting scary… can’t trust anything anymore  
1                     this dude is super unlikeable  
2                                  sick sick sick!   
3  such 

In [ ]:
# --- Custom Word-level Tokenizer ---
def word_tokenize_custom(text):
    text = str(text).lower()
    # Split by words (alphanumeric only)
    tokens = re.findall(r'\b\w+\b', text)
    return tokens

# --- Custom Sentence-level Tokenizer ---
def sentence_tokenize_custom(text):
    text = str(text)
    # Split by punctuation marks (., !, ?)
    sentences = re.split(r'(?<=[.!?]) +', text)
    return [s.strip() for s in sentences if s]

# Apply on dataset
df["Word_Tokens"] = df["Stemmed_Comments"].apply(word_tokenize_custom)
df["Sentence_Tokens"] = df["Comments"].apply(sentence_tokenize_custom)

# Show result
display(df.head(100))

,Comments,Reviews,Platform,Stemmed_Comments,No_Emoji_Comments,Word_Tokens,Sentence_Tokens
0,ai is geting scary… can’t trust anything anymore,negative,youtube,ai geting scary… can’t trust anything anymore,ai is geting scary… can’t trust anything anymore,"[ai, geting, scary, can, t, trust, anything, a...",[ai is geting scary… can’t trust anything anym...
1,this dude is super unlikeable,negative,youtube,dude super unlikeable,this dude is super unlikeable,"[dude, super, unlikeable]",[this dude is super unlikeable]
2,sick sick sick! 😡,negative,instagram,sick sick sick 😡,sick sick sick!,"[sick, sick, sick]","[sick sick sick!, 😡]"
3,"😑such an average movie, not worth the hype at all",negative,youtube,😑such average movie not worth hype all,"such an average movie, not worth the hype at all","[such, average, movie, not, worth, hype, all]","[😑such an average movie, not worth the hype at..."
4,nonsense...absolutely not true.,negative,instagram,nonsenseabsolutely not true,nonsense...absolutely not true.,"[nonsenseabsolutely, not, true]",[nonsense...absolutely not true.]
...,...,...,...,...,...,...,...
95,stop spreading rumors.,negative,twitter,stop spreading rumors,stop spreading rumors.,"[stop, spreading, rumors]",[stop spreading rumors.]
96,this is a terrible design flaw.,negative,instagram,terrible design flaw,this is a terrible design flaw.,"[terrible, design, flaw]",[this is a terrible design flaw.]
97,the server is lagging badly tonight.,negative,twitter,server lagging badly tonight,the server is lagging badly tonight.,"[server, lagging, badly, tonight]",[the server is lagging badly tonight.]
98,my order was cancelled without any notification.,negative,facebook,my order cancelled without any notification,my order was cancelled without any notification.,"[my, order, cancelled, without, any, notificat...",[my order was cancelled without any notificati...


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Initialize vectorizer
vectorizer = CountVectorizer()

# Fit and transform comments
X = vectorizer.fit_transform(df["Comments"].astype(str))

# Convert to DataFrame for readability
vectorized_df = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())

# Show first few rows
print(vectorized_df.head())

   10  11  15  2bhk  30  480p  5k  about  absolute  absolutely  ...  years  \
0   0   0   0     0   0     0   0      0         0           0  ...      0   
1   0   0   0     0   0     0   0      0         0           0  ...      0   
2   0   0   0     0   0     0   0      0         0           0  ...      0   
3   0   0   0     0   0     0   0      0         0           0  ...      0   
4   0   0   0     0   0     0   0      0         0           1  ...      0   

   yelling  yesterday  you  young  your  youth  yumm  zero  zoomed  
0        0          0    0      0     0      0     0     0       0  
1        0          0    0      0     0      0     0     0       0  
2        0          0    0      0     0      0     0     0       0  
3        0          0    0      0     0      0     0     0       0  
4        0          0    0      0     0      0     0     0       0  

[5 rows x 2427 columns]


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["Comments"].astype(str))


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features (X) and Labels (y)
X = df["Comments"].astype(str)
y = df["Reviews"].astype(str)

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

# ======================
# Models to Train
# ======================
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(kernel="linear"),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}

# Train & Evaluate Each Model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    print(f"\n===== {name} =====")
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))

# Find Best Model
best_model_name = max(results, key=results.get)
print("\n🔥 Best Model:", best_model_name, "with Accuracy:", results[best_model_name])


===== Naive Bayes =====
Accuracy: 0.7338709677419355
              precision    recall  f1-score   support

    negative       0.76      0.81      0.79        86
     neutral       0.89      0.35      0.51        68
    positive       0.68      0.94      0.79        94

    accuracy                           0.73       248
   macro avg       0.78      0.70      0.69       248
weighted avg       0.77      0.73      0.71       248


===== SVM =====
Accuracy: 0.7943548387096774
              precision    recall  f1-score   support

    negative       0.78      0.80      0.79        86
     neutral       0.75      0.59      0.66        68
    positive       0.83      0.94      0.88        94

    accuracy                           0.79       248
   macro avg       0.79      0.78      0.78       248
weighted avg       0.79      0.79      0.79       248


===== Decision Tree =====
Accuracy: 0.6370967741935484
              precision    recall  f1-score   support

    negative       0.60    

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features (X) and Labels (y)
X = df["Comments"].astype(str)
y = df["Reviews"].astype(str)

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

# ======================
# 2. Models to Train
# ======================
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(kernel="linear", probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}
trained_models = {}

# Train & Evaluate Each Model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    trained_models[name] = model
    print(f"\n===== {name} =====")
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))

# ======================
# 3. Pick Best Model
# ======================
best_model_name = max(results, key=results.get)
best_model = trained_models[best_model_name]

print("\n🔥 Best Model:", best_model_name, "with Accuracy:", results[best_model_name])

# ======================
# 4. Test the Best Model
# ======================
test_comments = [
    "I absolutely loved this movie, so emotional and inspiring!",
    "Worst product ever, total waste of money!",
    "It was okay, nothing too special.",
    "AI is becoming scary these days!"
]

# Convert test comments to vectors
X_new = vectorizer.transform(test_comments)

# Predict with best model
predictions = best_model.predict(X_new)

print("\n=== Predictions on New Comments ===")
for comment, label in zip(test_comments, predictions):
    print(f"{comment}  --->  {label}")


===== Naive Bayes =====
Accuracy: 0.7338709677419355
              precision    recall  f1-score   support

    negative       0.76      0.81      0.79        86
     neutral       0.89      0.35      0.51        68
    positive       0.68      0.94      0.79        94

    accuracy                           0.73       248
   macro avg       0.78      0.70      0.69       248
weighted avg       0.77      0.73      0.71       248


===== SVM =====
Accuracy: 0.7943548387096774
              precision    recall  f1-score   support

    negative       0.78      0.80      0.79        86
     neutral       0.75      0.59      0.66        68
    positive       0.83      0.94      0.88        94

    accuracy                           0.79       248
   macro avg       0.79      0.78      0.78       248
weighted avg       0.79      0.79      0.79       248


===== Decision Tree =====
Accuracy: 0.6370967741935484
              precision    recall  f1-score   support

    negative       0.60    

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Features (X) and Labels (y)
X = df["Comments"].astype(str)
y = df["Reviews"].astype(str)

# Improved Vectorizer
vectorizer = TfidfVectorizer(
    stop_words="english", ngram_range=(1,2), max_df=0.9, min_df=2
)
X_vec = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

# ======================
# 2. Models & Parameters
# ======================
param_grids = {
    "Naive Bayes": {
        "alpha": [0.5, 1.0, 1.5]
    },
    "SVM": {
        "C": [0.1, 1, 10],
        "kernel": ["linear", "rbf"]
    },
    "Decision Tree": {
        "max_depth": [None, 10, 20, 30],
        "min_samples_split": [2, 5, 10]
    },
    "Random Forest": {
        "n_estimators": [100, 200],
        "max_depth": [None, 10, 20],
        "max_features": ["sqrt", "log2"]
    }
}

models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(random_state=42)
}

results = {}
best_models = {}

# ======================
# 3. Train with GridSearch
# ======================
for name, model in models.items():
    print(f"\n🔍 Tuning {name} ...")
    grid = GridSearchCV(model, param_grids[name], cv=3, n_jobs=-1, verbose=1)
    grid.fit(X_train, y_train)

    best_models[name] = grid.best_estimator_
    y_pred = grid.best_estimator_.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc

    print(f"✅ Best Params for {name}: {grid.best_params_}")
    print(f"🎯 Accuracy: {acc}")
    print(classification_report(y_test, y_pred))

# ======================
# 4. Compare All Models
# ======================
print("\n📊 Accuracy Comparison:")
for name, acc in results.items():
    print(f"{name}: {acc:.4f}")

best_model_name = max(results, key=results.get)
print("\n🔥 Best Model:", best_model_name, "with Accuracy:", results[best_model_name])


🔍 Tuning Naive Bayes ...
Fitting 3 folds for each of 3 candidates, totalling 9 fits
✅ Best Params for Naive Bayes: {'alpha': 0.5}
🎯 Accuracy: 0.7298387096774194
              precision    recall  f1-score   support

    negative       0.74      0.74      0.74        86
     neutral       0.67      0.43      0.52        68
    positive       0.75      0.94      0.83        94

    accuracy                           0.73       248
   macro avg       0.72      0.70      0.70       248
weighted avg       0.72      0.73      0.71       248


🔍 Tuning SVM ...
Fitting 3 folds for each of 6 candidates, totalling 18 fits
✅ Best Params for SVM: {'C': 1, 'kernel': 'linear'}
🎯 Accuracy: 0.7379032258064516
              precision    recall  f1-score   support

    negative       0.75      0.73      0.74        86
     neutral       0.69      0.49      0.57        68
    positive       0.75      0.93      0.83        94

    accuracy                           0.74       248
   macro avg       0.73 

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# ======================
# 1. Load Dataset
# ======================
file_path = "Data Collection - Task 1 - Sheet1.csv"   # change path if needed
df = pd.read_csv(file_path)

# Clean dataset
df = df.dropna(how="all")
df.columns = df.columns.str.strip()

# Features (X) and Labels (y)
X = df["Comments"].astype(str)
y = df["Reviews"].astype(str)

# Vectorize using TF-IDF
vectorizer = TfidfVectorizer()
X_vec = vectorizer.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_vec, y, test_size=0.2, random_state=42
)

# ======================
# 2. Models to Train
# ======================
models = {
    "Naive Bayes": MultinomialNB(),
    "SVM": SVC(kernel="linear", probability=True),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42)
}

results = {}
trained_models = {}

# Train & Evaluate Each Model
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results[name] = acc
    trained_models[name] = model
    print(f"\n===== {name} =====")
    print("Accuracy:", acc)
    print(classification_report(y_test, y_pred))

# ======================
# 3. Pick Best Model
# ======================
best_model_name = max(results, key=results.get)
best_model = trained_models[best_model_name]

print("\n🔥 Best Model:", best_model_name, "with Accuracy:", results[best_model_name])

# ======================
# 4. Take User Input and Predict
# ======================
while True:
    user_input = input("\nEnter a comment (or type 'exit' to quit): ")
    if user_input.lower() == "exit":
        break
    X_new = vectorizer.transform([user_input])
    prediction = best_model.predict(X_new)[0]
    print(f"👉 Predicted Review: {prediction}")



===== Naive Bayes =====
Accuracy: 0.7459677419354839
              precision    recall  f1-score   support

    Negative       0.80      0.83      0.81        86
     Neutral       0.79      0.40      0.53        65
    Positive       0.70      0.91      0.79        97

    accuracy                           0.75       248
   macro avg       0.76      0.71      0.71       248
weighted avg       0.76      0.75      0.73       248


===== SVM =====
Accuracy: 0.7620967741935484
              precision    recall  f1-score   support

    Negative       0.76      0.76      0.76        86
     Neutral       0.66      0.62      0.63        65
    Positive       0.82      0.87      0.84        97

    accuracy                           0.76       248
   macro avg       0.75      0.75      0.75       248
weighted avg       0.76      0.76      0.76       248


===== Decision Tree =====
Accuracy: 0.6451612903225806
              precision    recall  f1-score   support

    Negative       0.65    